<a href="https://colab.research.google.com/github/elangbijak4/multi-agent-AI/blob/main/Simple_Respon_No_Trap_in_Looping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# BLOCK 1: Fixed ORIGINAL Simple Reflex Agent (anti-oscillation + random tie-break)
# Run in Google Colab / Jupyter.

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, patches
from IPython.display import HTML, display
import random

# Parameters
GRID_ROWS = 5
GRID_COLS = 7
DIRTY_PROB = 0.25
STEPS = 120
np.random.seed(42)
random.seed(42)

# Environment
env = np.zeros((GRID_ROWS, GRID_COLS), dtype=int)
env[np.random.rand(GRID_ROWS, GRID_COLS) < DIRTY_PROB] = 1
env[1, 3] = 2
env[3, 5] = 2

# Agent
agent_pos = [0, 0]
last_pos = None  # to avoid immediate backtracking

def simple_reflex_action_no_osc(perception, position, last_position):
    r, c = position
    # If dirty, clean
    if perception == 1:
        return 'SUCK', (r, c)
    # Generate sweep candidate
    candidates = []
    if r % 2 == 0:
        candidates.append((r, c+1))
    else:
        candidates.append((r, c-1))
    candidates.extend([(r+1, c), (r-1, c), (r, c+1), (r, c-1)])
    # Filter
    valid = []
    for pr, pc in candidates:
        if 0 <= pr < GRID_ROWS and 0 <= pc < GRID_COLS and env[pr, pc] != 2:
            if (pr, pc) != last_position:  # avoid immediate backtracking if possible
                valid.append((pr, pc))
    # If no valid excluding last_position, allow last_position
    if not valid:
        for pr, pc in candidates:
            if 0 <= pr < GRID_ROWS and 0 <= pc < GRID_COLS and env[pr, pc] != 2:
                valid.append((pr, pc))
    if not valid:
        return 'NO_OP', (r, c)
    # Prefer move to cell that is dirty (if adjacent)
    for pos in valid:
        if env[pos[0], pos[1]] == 1:
            return 'MOVE', pos
    # Otherwise choose random among valids to break symmetry and avoid loops
    choice = random.choice(valid)
    return 'MOVE', choice

# Simulation
positions, actions, env_history = [], [], []
for t in range(STEPS):
    positions.append(tuple(agent_pos))
    env_history.append(env.copy())
    perception = env[agent_pos[0], agent_pos[1]]
    action, new_pos = simple_reflex_action_no_osc(perception, tuple(agent_pos), last_pos)
    actions.append(action)
    if action == 'SUCK':
        env[agent_pos[0], agent_pos[1]] = 0
    elif action == 'MOVE':
        last_pos = tuple(agent_pos)
        agent_pos = [new_pos[0], new_pos[1]]
    else:
        last_pos = tuple(agent_pos)

positions.append(tuple(agent_pos))
env_history.append(env.copy())

# Visualization
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_xticks(np.arange(-0.5, GRID_COLS, 1))
ax.set_yticks(np.arange(-0.5, GRID_ROWS, 1))
ax.set_xticklabels([]); ax.set_yticklabels([])
ax.grid(True)

def draw_frame(i):
    ax.clear()
    ax.set_title(f"Step {i}: Action = {actions[i] if i < len(actions) else '-'}")
    ax.set_xticks(np.arange(-0.5, GRID_COLS, 1))
    ax.set_yticks(np.arange(-0.5, GRID_ROWS, 1))
    ax.set_xticklabels([]); ax.set_yticklabels([])
    ax.grid(True)
    grid = env_history[i]
    for r in range(GRID_ROWS):
        for c in range(GRID_COLS):
            cell = grid[r, c]
            if cell == 1:
                ax.text(c, r, "●", fontsize=14, ha='center', va='center')
            elif cell == 2:
                ax.add_patch(patches.Rectangle((c-0.3, r-0.3), 0.6, 0.6, linewidth=1))
            else:
                ax.add_patch(patches.Rectangle((c-0.5, r-0.5), 1, 1, linewidth=0.2, fill=False))
    pr, pc = positions[i]
    ax.plot(pc, pr, marker='o', markersize=18)
    ax.text(pc, pr-0.35, "Agent", ha='center')

ani = animation.FuncAnimation(fig, draw_frame, frames=len(actions), interval=350, repeat=False)
plt.close(fig)
display(HTML(ani.to_jshtml()))

print("Action transcript (first 40 steps):")
for t, act in enumerate(actions[:40]):
    print(f"Step {t:2d}: Pos {positions[t]} -> {act}")


Action transcript (first 40 steps):
Step  0: Pos (0, 0) -> MOVE
Step  1: Pos (0, 1) -> MOVE
Step  2: Pos (0, 2) -> MOVE
Step  3: Pos (0, 3) -> MOVE
Step  4: Pos (0, 4) -> SUCK
Step  5: Pos (0, 4) -> MOVE
Step  6: Pos (0, 5) -> SUCK
Step  7: Pos (0, 5) -> MOVE
Step  8: Pos (0, 6) -> SUCK
Step  9: Pos (0, 6) -> MOVE
Step 10: Pos (1, 6) -> SUCK
Step 11: Pos (1, 6) -> MOVE
Step 12: Pos (1, 5) -> MOVE
Step 13: Pos (0, 5) -> MOVE
Step 14: Pos (0, 6) -> MOVE
Step 15: Pos (1, 6) -> MOVE
Step 16: Pos (1, 5) -> MOVE
Step 17: Pos (1, 4) -> MOVE
Step 18: Pos (2, 4) -> MOVE
Step 19: Pos (2, 3) -> MOVE
Step 20: Pos (3, 3) -> MOVE
Step 21: Pos (4, 3) -> SUCK
Step 22: Pos (4, 3) -> MOVE
Step 23: Pos (4, 4) -> SUCK
Step 24: Pos (4, 4) -> MOVE
Step 25: Pos (4, 5) -> MOVE
Step 26: Pos (4, 6) -> MOVE
Step 27: Pos (3, 6) -> MOVE
Step 28: Pos (2, 6) -> MOVE
Step 29: Pos (1, 6) -> MOVE
Step 30: Pos (1, 5) -> MOVE
Step 31: Pos (2, 5) -> MOVE
Step 32: Pos (2, 4) -> MOVE
Step 33: Pos (2, 3) -> MOVE
Step 34: Pos